In [ ]:
! pip install googlesearch-python langchain pydantic requests


In [ ]:
!pip install langchain

In [ ]:
!pip install duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.0 MB/s eta 0:00:00


In [ ]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [ ]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 3.6 MB/s eta 0:00:00


In [ ]:
import os
import json
from typing import List
from pydantic import BaseModel, Field
import time
# Import LangChain components:
# from langchain.chat_models import ChatOpenAI
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.output_parsers import PydanticOutputParser




In [ ]:
import re
import json
from typing import List
from pydantic import BaseModel, Field

# Define your structured output using Pydantic.
class MoviesList(BaseModel):
    movies: List[str] = Field(
        ...,
        description="A list of 100 movie names related to the provided movie description."
    )

# Set up a Pydantic parser from the output schema above.
output_parser = PydanticOutputParser(pydantic_object=MoviesList)

# Define the prompt template for generating the movie list.
prompt_template = """You are provided with a concise movie description and the results of a DuckDuckGo web search.
Movie Description:
{movie_description}

DuckDuckGo Search Results:
{search_results}

Based on the above information, list 100 movie names that are most relevant or similar to the provided description.
Output only a JSON object with a single key "movies" whose value is an array of movie name strings.
Do not include any commentary or additional text outside the JSON.
Your output format should follow these instructions exactly:
{output_format}
"""

# Insert instructions for the output using the parser.
output_format = output_parser.get_format_instructions()

# Create the final prompt template.
final_prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["movie_description", "search_results"],
    partial_variables={"output_format": output_format}
)

# Define a prompt template for summarizing long movie descriptions.
summarization_prompt = """The following movie description is too long. Make sure the paragraph does not exceed 150 words.
Make sure that the summarized version could be searched in the DuckDuckGo search tool.
Please provide a concise summary (in one paragraph) that retains the essential details:
{movie_description}
"""
summarization_template = PromptTemplate(
    template=summarization_prompt,
    input_variables=["movie_description"]
)

def main(movie_description: str):
    # Initialize the language model 
    llm = ChatGroq(temperature=0.5, groq_api_key="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", model_name="gemma2-9b-it")

    # Summarize the movie description if it is too long (threshold: 150 words).
    if len(movie_description.split()) > 150:
        summarization_chain = LLMChain(llm=llm, prompt=summarization_template)
        summarized_description = summarization_chain.run(movie_description=movie_description)
        movie_description = summarized_description.strip()
        print("Summarized Movie Description:", movie_description)
        print("Length of Summarized Movie Description:", len(movie_description.split()))

    # Create an instance of the DuckDuckGo search tool.
    duckduckgo = DuckDuckGoSearchRun()

    # Split the movie description into individual sentences.
    # This regex splits on punctuation followed by whitespace.
    sentences = re.split(r'(?<=[.!?])\s+', movie_description)
    aggregated_search_results = ""

    # Iterate over each sentence and perform a search.
    for sentence in sentences:
        # Construct a query tailored to the individual sentence.
        query = f"Movie description: '{sentence}' - want List similar movie names."
        time.sleep(2)
        sentence_result = duckduckgo.invoke(query)
        aggregated_search_results += sentence_result + "\n"

    print("Aggregated Search Results:\n", aggregated_search_results)

    # Create the main LLM chain to process the full description and the aggregated search results.
    chain = LLMChain(llm=llm, prompt=final_prompt)

    # Generate the raw output containing the JSON.
    raw_result = chain.run(movie_description=movie_description, search_results=aggregated_search_results)

    # Parse the output using the Pydantic parser.
    try:
        movies_list = output_parser.parse(raw_result)
    except Exception as e:
        print("Error parsing output:", e)
        movies_list = MoviesList(movies=[])

    # Pretty-print the final structured JSON output.
    print(json.dumps(movies_list.dict(), indent=4))

In [ ]:
if __name__ == "__main__":
    description = (
        "Ok, so my hubby, brother-in-law, and I rented a movie from redbox in the mid 2000’s (around the time streaming was becoming big) and I gotta say, we were glad we only paid a buck for it. This movie was so bad we talked and joked through the entire thing like the robots out of MST3K. But it bothers me that none of us can remember the name of the movie or what it was about. The only thing any of us really remember was it took place is completely or mostly in a third world country and near the end this woman, American I think, was standing either topless or naked in a bunch of mud and was surrounded by young boys. Most of whom were somewhere around 12-14 years old. And they start covering her in mud and as they do so, they are groping her breasts and ass. Not sure if they were supposed to, but they were. I think this scene stuck with us because it was about the best part of the movie and the only thing even relatively exciting that happened. We kept watching hoping that it would get better at some point. That someone didn’t actually, from start to finish, make a movie that terrible. I hope someone can help, because I would really like that know what that movie was supposed to be about and what it was called. I thank you all in advance"
    )
    main(description)

Summarized Movie Description: Three friends are trying to identify a poorly-made movie they rented in the mid-2000s.  The film, set primarily in a developing country, featured a scene where an American woman was covered in mud by a group of preteen boys who groped her. This unsettling scene was the most memorable part of the otherwise forgettable film.
Length of Summarized Movie Description: 55
Aggregated Search Results:
 If you know the actors but not their names, describe their other famous roles. 3. Click the "Guess" button, and the AI will analyze your clues and present the most likely movie match. 4. If the result isn't quite right, you can refine your description or click the 'Try Again' button until you find the movie you're looking for. Pros What Is My Movie is a super-accurate movie-finder site that can suggest the name of a movie by a scene or plot description, an incomplete title, and actors' and directors' names. By "super-accurate", we mean that the site can easily find a 

<ipython-input-48-51d8b3a9ff06>:99: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(json.dumps(movies_list.dict(), indent=4))
